In [17]:
import pandas as pd
import numpy as np
from pathlib import Path

pd.set_option("display.max_columns", 200)

In [18]:
# Define relative paths to data files
ROOT = Path("..")
WELLS_PATH = ROOT / "data" / "raw" / "wells.csv"

WELLS_PATH.exists(), WELLS_PATH

(True, WindowsPath('../data/raw/wells.csv'))

In [19]:
# Load wells data and display its shape and columns
wells = pd.read_csv(WELLS_PATH)
wells.shape, wells.columns.tolist()
#wells.head()

((1082, 25),
 ['DataID',
  'ActcualDepth',
  'Depth_IDW',
  'DepthType',
  'Frequency',
  'WellDepth',
  'Depth',
  'xCoord',
  'yCoord',
  'City',
  'State',
  'ZipCode',
  'Collection',
  'Sampling_P',
  'GelogicUnit',
  'Rocks',
  'Groups',
  'FORMATION',
  'ArsenicOld',
  'ArsenicCat',
  'ArsenicCat2',
  'pH',
  'Belt',
  'Township',
  'RockType'])

In [20]:
# sample data for testing when building the pipeline
USE_SAMPLE = False
SAMPLE_N = 20

if USE_SAMPLE:
    wells = wells.sample(SAMPLE_N, random_state=42).copy()
    wells.shape

In [21]:
# take a look at the data
wells.head(10)

,DataID,ActcualDepth,Depth_IDW,DepthType,Frequency,WellDepth,Depth,xCoord,yCoord,City,State,ZipCode,Collection,Sampling_P,GelogicUnit,Rocks,Groups,FORMATION,ArsenicOld,ArsenicCat,ArsenicCat2,pH,Belt,Township,RockType
0,1,205.0,205.003498,Actual,1,205,151 to 300,-81.21728,35.34246,Dallas,NC,28034,5/2/2016,Outside spigot,Massive to weakly foliated granitic rocks,granite,Intrusive Rocks,Foliated to massive granitic rock,< 0.005,0,Below,6.1,Kings Mountain Belt,DALLAS,1
1,2,NaN,304.875953,Imputed,1,305,> 300,-81.21802,35.31511,Dallas,NC,28034,10/7/2014,Well,Massive to weakly foliated granitic rocks,granite,Intrusive Rocks,Foliated to massive granitic rock,< 0.005,0,Below,6.4,Kings Mountain Belt,DALLAS,1
2,3,NaN,414.922922,Imputed,1,415,> 300,-81.26592,35.39481,Lincolnton,NC,28092,2/4/2015,Outside spigot,Stratified metamorphic rocks,mica schist & quartzite,Metamorphic rocks,Mica schist,0.05,1,Above,7.9,Inner Piedmont,CHERRYVILLE,5
3,4,188.0,188.001758,Actual,1,188,151 to 300,-81.26164,35.35970,Bessemer City,NC,28106,10/24/2011,Well,Stratified metamorphic rocks,schist & phyllite,Metamorphic rocks,Blacksburg Formation,0.006,1,Above,0.0,Kings Mountain Belt,CHERRYVILLE,5
4,5,705.0,704.988274,Actual,2,705,> 300,-81.01120,35.16907,Belmont,NC,28012,8/27/2012,Well,Metamorphosed quartz diorite,metamorphic rock,Intrusive Rocks,Metamorphosed quartz diorite,< 0.005,0,Below,8.0,Charlotte Belt,SOUTH POINT,4
5,6,NaN,201.731697,Imputed,1,202,151 to 300,-81.34885,35.35283,Cherryville,NC,28021,11/5/2012,Outside spigot,Massive to weakly foliated granitic rocks,granite & pegmatite,Intrusive Rocks,Cherryville Granite,< 0.005,0,Below,6.3,Inner Piedmont,CHERRYVILLE,1
6,7,NaN,357.960637,Imputed,2,358,> 300,-81.24654,35.18709,Gastonia,NC,28052,6/26/2017,Well head,Massive to weakly foliated granitic rocks,granite,Intrusive Rocks,Foliated to massive granitic rock,< 0.005,0,Below,7.6,Kings Mountain Belt,GASTONIA,1
7,8,185.0,184.999714,Actual,2,185,151 to 300,-81.11587,35.27869,Gastonia,NC,28054,4/17/2012,Well,Stratified metamorphic rocks,schist & metavolcanic rock,Metamorphic rocks,Battleground Formation,< 0.005,0,Below,0.0,Kings Mountain Belt,GASTONIA,5
8,9,305.0,302.390338,Actual,1,305,> 300,-81.27669,35.41236,Lincolnton,NC,28092,9/14/2011,Well,Stratified metamorphic rocks,mica schist & quartzite,Metamorphic rocks,Mica schist,0.006,1,Above,0.0,Inner Piedmont,CHERRYVILLE,5
9,10,NaN,441.827265,Imputed,1,442,> 300,-81.18383,35.33568,Dallas,NC,28034,3/24/2011,Outside spigot,Massive to weakly foliated granitic rocks,granite,Intrusive Rocks,Foliated to massive granitic rock,< 0.005,0,Below,8.0,Kings Mountain Belt,DALLAS,1


In [22]:
# check for missingness and class balance 
wells.isna().mean().sort_values(ascending=False).head(20)
wells["ArsenicCat"].value_counts(dropna=False)



ArsenicCat
0    994
1     88
Name: count, dtype: int64

In [23]:
# lets do a schema check see what columns we have
REQUIRED = [
    "DataID",
    "xCoord", "yCoord",
    "pH",
    "ActcualDepth", "Depth_IDW", "DepthType",
    "FORMATION",
    "ArsenicCat", "ArsenicCat2", "ArsenicOld",
    "WellDepth",
]
[c for c in REQUIRED if c not in wells.columns]


[]

In [24]:
# for the pH value, treat 0 and out of range values as missing
wells["pH"] = pd.to_numeric(wells["pH"], errors="coerce")
wells.loc[(wells["pH"] <= 0) | (wells["pH"] > 14), "pH"] = np.nan
wells["pH"].describe()


count    990.000000
mean       7.073838
std        0.539251
min        5.100000
25%        6.700000
50%        7.100000
75%        7.400000
max        9.700000
Name: pH, dtype: float64

In [25]:
# well depth - use actual measured depth, if unavailable use the interpolated depth from inverse distance weighting (IDW)
wells["ActcualDepth"] = pd.to_numeric(wells["ActcualDepth"], errors="coerce")
wells["Depth_IDW"] = pd.to_numeric(wells["Depth_IDW"], errors="coerce")
wells["WellDepth"] = pd.to_numeric(wells["WellDepth"], errors="coerce")

wells["depth_value"] = wells["ActcualDepth"]
wells.loc[wells["depth_value"].isna(), "depth_value"] = wells.loc[wells["depth_value"].isna(), "Depth_IDW"]
wells.loc[wells["depth_value"].isna(), "depth_value"] = wells.loc[wells["depth_value"].isna(), "WellDepth"]

wells["depth_value"].describe()


count    1082.000000
mean      265.596958
std       116.528147
min        15.000000
25%       200.000000
50%       244.139382
75%       306.552248
max       800.000000
Name: depth_value, dtype: float64

In [26]:
# create a value to help with exploratory plots
def parse_arsenic_value(v):
    if pd.isna(v):
        return np.nan
    s = str(v).strip().replace(",", "")
    if s.startswith("<"):
        s = s[1:].strip()
    try:
        return float(s)
    except:
        return np.nan
    
wells["arsenic_value"] = wells["ArsenicOld"].apply(parse_arsenic_value)
wells[["ArsenicOld", "arsenic_value"]].head(10)

,ArsenicOld,arsenic_value
0,< 0.005,0.005
1,< 0.005,0.005
2,0.05,0.050
3,0.006,0.006
4,< 0.005,0.005
5,< 0.005,0.005
6,< 0.005,0.005
7,< 0.005,0.005
8,0.006,0.006
9,< 0.005,0.005


In [27]:
# Create a binary target used in modeling.
wells["y"] = pd.to_numeric(wells["ArsenicCat"], errors="coerce").astype("Int64")
wells["y"].value_counts(dropna=False)


y
0    994
1     88
Name: count, dtype: Int64

In [28]:
# rule for dealing with dups - for each location keep the highest arsenic measurement
wells["xCoord"] = pd.to_numeric(wells["xCoord"], errors="coerce")
wells["yCoord"] = pd.to_numeric(wells["yCoord"], errors="coerce")

wells["loc_key"] = wells["xCoord"].round(6).astype(str) + "_" + wells["yCoord"].round(6).astype(str)
wells["_ars_sort"] = wells["arsenic_value"].fillna(-1)

wells_dedup = (
    wells.sort_values(["loc_key", "_ars_sort", "y"], ascending=[True, False, False])
         .drop_duplicates("loc_key", keep="first")
         .drop(columns=["_ars_sort"])
)

wells.shape, wells_dedup.shape


((1082, 30), (1082, 29))

In [29]:
# Build the minimal modeling table
model_df = wells_dedup[[
    "DataID",
    "xCoord", "yCoord",
    "pH",
    "depth_value",
    "DepthType",
    "FORMATION",
    "ArsenicCat2",
    "arsenic_value",
    "y",
]].copy()

model_df.rename(columns={
    "DataID": "well_id",
    "xCoord": "longitude",
    "yCoord": "latitude",
    "pH": "ph",
    "depth_value": "depth",
    "DepthType": "depth_type",
    "FORMATION": "geology_unit",
    "ArsenicCat2": "EPA_threshold",
}, inplace=True)

model_df.head(10)


,well_id,longitude,latitude,ph,depth,depth_type,geology_unit,EPA_threshold,arsenic_value,y
984,985,-80.94120,35.35403,6.9,185.000000,Actual,Metamorphosed quartz diorite,Below,0.005,0
231,232,-80.94185,35.35463,6.7,122.522975,Imputed,Metamorphosed quartz diorite,Below,0.005,0
518,519,-80.94308,35.35391,6.9,95.000000,Actual,Metamorphosed quartz diorite,Below,0.005,0
75,76,-80.94451,35.35331,NaN,705.000000,Actual,Metamorphosed quartz diorite,Below,0.005,0
486,487,-80.94483,35.35314,7.3,395.482890,Imputed,Metamorphosed quartz diorite,Below,0.005,0
710,711,-80.94654,35.35396,7.7,185.000000,Actual,Metamorphosed quartz diorite,Below,0.005,0
477,478,-80.94807,35.35414,6.7,230.563790,Imputed,Metamorphosed quartz diorite,Below,0.005,0
902,903,-80.95019,35.35441,7.1,553.494475,Imputed,Metamorphosed quartz diorite,Below,0.005,0
402,403,-80.97330,35.40034,6.9,362.655005,Imputed,Metamorphosed quartz diorite,Below,0.005,0
561,562,-80.97657,35.39098,7.7,315.211841,Imputed,Metamorphosed quartz diorite,Below,0.005,0


In [30]:
# class balance in the raw data without droping the missing rows
wells["y"].value_counts(dropna=False)

y
0    994
1     88
Name: count, dtype: Int64

In [31]:
# final data clean up - drop rows missing the predictors and target
core = ["well_id", "longitude", "latitude", "ph", "depth", "geology_unit", "EPA_threshold", "y"]
before = len(model_df)
model_df_clean = model_df.dropna(subset=core).copy()
model_df_clean["y"].value_counts(dropna=False)
after = len(model_df_clean)

(before, after), model_df_clean["y"].value_counts(dropna=False)


((1082, 990),
 y
 0    912
 1     78
 Name: count, dtype: Int64)

In [32]:
out_dir = ROOT / "data" / "processed"
out_dir.mkdir(parents=True, exist_ok=True)

out_path = out_dir / "model_table.csv"
model_df_clean.to_csv(out_path, index=False)

out_path

WindowsPath('../data/processed/model_table.csv')